### Titanic survival
#### Binary classification with sklearn

Data source : https://www.kaggle.com/c/titanic

Variables:

 - survival:	Survival	                                   0 = No, 1 = Yes
 - pclass:       Ticket class	                               1 = 1st, 2 = 2nd, 3 = 3rd
 - sex:	        Sex	Male/Female
 - Age:	     Age in years	
 - sibsp:	    # of siblings / spouses aboard the Titanic	
 - parch:	    # of parents / children aboard the Titanic	
 - ticket:	    Ticket number	
 - fare:	    Passenger fare	
 - cabin:	    Cabin number	
 - embarked:	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton
 
 Survival is the variable to be predicted

In [1]:
import pandas as pd
import sklearn as sk
import numpy as np

In [2]:
data = pd.read_csv('train.csv')

### Explore the data

In [3]:
len(data)

891

In [4]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [6]:
data.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

### Check for missing/empty values

In [7]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Since we only have 891 samples to begin with, we cannot simply drop the rows with na values. The cabin data looks quite intriguing and could very vell have a relationship to the predicted variable

In [8]:
data['Cabin'].dropna().head()

1      C85
3     C123
6      E46
10      G6
11    C103
Name: Cabin, dtype: object

For now, we'll discard this variable however because of large number of missing values

In [9]:
data = data.drop('Cabin', axis=1)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


While we're at it, lets discard some other variables as well

In [10]:
data = data.drop(['PassengerId', 'Name', 'Ticket'], axis=1)

Hence, our final task will be to fit a binary classification model:

(Pclass, Sex, Age, SibSp, Parch, Fare, Embarked) --> Survived

Before that, we still need to handle the NaN values in Age and Embarked

In [11]:
#fill the missing values with mean age
data['Age'] = data['Age'].fillna(data['Age'].mean())

#Embarked has only 2 missing values, we will simply discard those rows
data = data.dropna(subset=['Embarked'])

Expect 891 - 2 = 889

In [12]:
len(data)

889

In [13]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [14]:
data.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

Looking good!

### Handle the categorical variables

We treat integer values such as number of siblings or parents/children as categorical values

In [15]:
cat_feats = ['Pclass',
            'Sex',
            'SibSp',
            'Parch',
            'Embarked']
num_feats = ['Age', 'Fare']


Explicitly cast everything to string in order to do one-hot encoding 

In [16]:
data[cat_feats] = data[cat_feats].astype('object')
data[cat_feats].dtypes

Pclass      object
Sex         object
SibSp       object
Parch       object
Embarked    object
dtype: object

In [17]:
data_cat_onehot = pd.get_dummies(data[cat_feats])
data_cat_onehot.head()

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,...,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,0,0,1,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,1,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,0,0,1,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


A little bit of a hack to make training data to same dimensions with test data

In [18]:
data_cat_onehot.insert(19, 'Parch_9', value=0)

In [19]:
list(data_cat_onehot)

['Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'Sex_female',
 'Sex_male',
 'SibSp_0',
 'SibSp_1',
 'SibSp_2',
 'SibSp_3',
 'SibSp_4',
 'SibSp_5',
 'SibSp_8',
 'Parch_0',
 'Parch_1',
 'Parch_2',
 'Parch_3',
 'Parch_4',
 'Parch_5',
 'Parch_6',
 'Parch_9',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

Build the final dataset by dataframe concatenation

In [20]:
data[num_feats].head()

,Age,Fare
0,22.0,7.2500
1,38.0,71.2833
2,26.0,7.9250
3,35.0,53.1000
4,35.0,8.0500


In [21]:
X = pd.concat([data[num_feats], data_cat_onehot], axis=1)
y = data['Survived']

In [22]:
X.head()

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,...,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,38.0,71.2833,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,26.0,7.9250,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,35.0,53.1000,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,35.0,8.0500,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [23]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


### Feature normalization

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
sc = StandardScaler()

In [26]:
X[:] = sc.fit_transform(X[:])

In [27]:
X.head()

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,...,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9,Embarked_C,Embarked_Q,Embarked_S
0,-0.590495,-0.500240,-0.563060,-0.510875,0.900328,-0.735342,0.735342,-1.463333,1.803771,-0.180334,...,-0.391213,-0.314464,-0.075207,-0.067229,-0.075207,-0.033558,0.0,-0.482711,-0.307941,0.616794
1,0.643971,0.788947,1.776008,-0.510875,-1.110706,1.359911,-1.359911,-1.463333,1.803771,-0.180334,...,-0.391213,-0.314464,-0.075207,-0.067229,-0.075207,-0.033558,0.0,2.071634,-0.307941,-1.621287
2,-0.281878,-0.486650,-0.563060,-0.510875,0.900328,1.359911,-1.359911,0.683372,-0.554394,-0.180334,...,-0.391213,-0.314464,-0.075207,-0.067229,-0.075207,-0.033558,0.0,-0.482711,-0.307941,0.616794
3,0.412509,0.422861,1.776008,-0.510875,-1.110706,1.359911,-1.359911,-1.463333,1.803771,-0.180334,...,-0.391213,-0.314464,-0.075207,-0.067229,-0.075207,-0.033558,0.0,-0.482711,-0.307941,0.616794
4,0.412509,-0.484133,-0.563060,-0.510875,0.900328,-0.735342,0.735342,0.683372,-0.554394,-0.180334,...,-0.391213,-0.314464,-0.075207,-0.067229,-0.075207,-0.033558,0.0,-0.482711,-0.307941,0.616794


### Train / test split

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
X_train.shape

(622, 25)

In [31]:
y_train.shape

(622,)

In [32]:
X_test.shape

(267, 25)

In [33]:
y_test.shape

(267,)

### Define and train the model 

In [34]:
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn.svm import SVC
from sklearn.externals import joblib
import datetime as dt

In [35]:
#logreg = linear_model.LogisticRegression()
#logreg = naive_bayes.GaussianNB()
#logreg = SVC(C=0.5, kernel='poly', degree=3, verbose=True)

While a simple logistic regression will actually yield quite satisfactory results (f1-score 0.81 in the test set) but just to make things a bit more interesting, we'll introduce a Support Vector Machine and do a gridsearch over the hyperparameter space

In [36]:
c_grid = list(np.linspace(0.001, 1, num = 100))
g_grid = list(np.linspace(0.0004, 4, num = 100))

for c in c_grid:
    for g in g_grid:
        
        logreg = SVC(C=c, kernel='rbf', degree=3, verbose=True, gamma=g)
        logreg.fit(X_train, y_train)
        print('Training complete with c: ', c, 'gamma: ', g)
    
        now = str(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        filename = f'models/trainedmodel_c_{c}_g_{g}.pkl'
        joblib.dump(logreg, filename)
    
        print("Model saved as ", filename)
    

[LibSVM]Training complete with c:  0.001 gamma:  0.0004
Model saved as  models/trainedmodel_c_0.001_g_0.0004.pkl
[LibSVM]Training complete with c:  0.001 gamma:  0.040799999999999996
Model saved as  models/trainedmodel_c_0.001_g_0.040799999999999996.pkl
[LibSVM]Training complete with c:  0.001 gamma:  0.0812
Model saved as  models/trainedmodel_c_0.001_g_0.0812.pkl
[LibSVM]Training complete with c:  0.001 gamma:  0.1216
Model saved as  models/trainedmodel_c_0.001_g_0.1216.pkl
[LibSVM]Training complete with c:  0.001 gamma:  0.162
Model saved as  models/trainedmodel_c_0.001_g_0.162.pkl
[LibSVM]Training complete with c:  0.001 gamma:  0.2024
Model saved as  models/trainedmodel_c_0.001_g_0.2024.pkl
[LibSVM]Training complete with c:  0.001 gamma:  0.24280000000000002
Model saved as  models/trainedmodel_c_0.001_g_0.24280000000000002.pkl
[LibSVM]Training complete with c:  0.001 gamma:  0.2832
Model saved as  models/trainedmodel_c_0.001_g_0.2832.pkl
[LibSVM]Training complete with c:  0.001 gam

Model saved as  models/trainedmodel_c_0.001_g_3.0707999999999998.pkl
[LibSVM]Training complete with c:  0.001 gamma:  3.1111999999999997
Model saved as  models/trainedmodel_c_0.001_g_3.1111999999999997.pkl
[LibSVM]Training complete with c:  0.001 gamma:  3.1515999999999997
Model saved as  models/trainedmodel_c_0.001_g_3.1515999999999997.pkl
[LibSVM]Training complete with c:  0.001 gamma:  3.1919999999999997
Model saved as  models/trainedmodel_c_0.001_g_3.1919999999999997.pkl
[LibSVM]Training complete with c:  0.001 gamma:  3.2323999999999997
Model saved as  models/trainedmodel_c_0.001_g_3.2323999999999997.pkl
[LibSVM]Training complete with c:  0.001 gamma:  3.2727999999999997
Model saved as  models/trainedmodel_c_0.001_g_3.2727999999999997.pkl
[LibSVM]Training complete with c:  0.001 gamma:  3.3131999999999997
Model saved as  models/trainedmodel_c_0.001_g_3.3131999999999997.pkl
[LibSVM]Training complete with c:  0.001 gamma:  3.3535999999999997
Model saved as  models/trainedmodel_c_0.0

[LibSVM]Training complete with c:  0.011090909090909092 gamma:  1.6971999999999998
Model saved as  models/trainedmodel_c_0.011090909090909092_g_1.6971999999999998.pkl
[LibSVM]Training complete with c:  0.011090909090909092 gamma:  1.7375999999999998
Model saved as  models/trainedmodel_c_0.011090909090909092_g_1.7375999999999998.pkl
[LibSVM]Training complete with c:  0.011090909090909092 gamma:  1.7779999999999998
Model saved as  models/trainedmodel_c_0.011090909090909092_g_1.7779999999999998.pkl
[LibSVM]Training complete with c:  0.011090909090909092 gamma:  1.8183999999999998
Model saved as  models/trainedmodel_c_0.011090909090909092_g_1.8183999999999998.pkl
[LibSVM]Training complete with c:  0.011090909090909092 gamma:  1.8587999999999998
Model saved as  models/trainedmodel_c_0.011090909090909092_g_1.8587999999999998.pkl
[LibSVM]Training complete with c:  0.011090909090909092 gamma:  1.8991999999999998
Model saved as  models/trainedmodel_c_0.011090909090909092_g_1.8991999999999998.pk

Model saved as  models/trainedmodel_c_0.011090909090909092_g_3.8788.pkl
[LibSVM]Training complete with c:  0.011090909090909092 gamma:  3.9192
Model saved as  models/trainedmodel_c_0.011090909090909092_g_3.9192.pkl
[LibSVM]Training complete with c:  0.011090909090909092 gamma:  3.9596
Model saved as  models/trainedmodel_c_0.011090909090909092_g_3.9596.pkl
[LibSVM]Training complete with c:  0.011090909090909092 gamma:  4.0
Model saved as  models/trainedmodel_c_0.011090909090909092_g_4.0.pkl
[LibSVM]Training complete with c:  0.021181818181818184 gamma:  0.0004
Model saved as  models/trainedmodel_c_0.021181818181818184_g_0.0004.pkl
[LibSVM]Training complete with c:  0.021181818181818184 gamma:  0.040799999999999996
Model saved as  models/trainedmodel_c_0.021181818181818184_g_0.040799999999999996.pkl
[LibSVM]Training complete with c:  0.021181818181818184 gamma:  0.0812
Model saved as  models/trainedmodel_c_0.021181818181818184_g_0.0812.pkl
[LibSVM]Training complete with c:  0.02118181818

Model saved as  models/trainedmodel_c_0.021181818181818184_g_2.1012.pkl
[LibSVM]Training complete with c:  0.021181818181818184 gamma:  2.1416
Model saved as  models/trainedmodel_c_0.021181818181818184_g_2.1416.pkl
[LibSVM]Training complete with c:  0.021181818181818184 gamma:  2.182
Model saved as  models/trainedmodel_c_0.021181818181818184_g_2.182.pkl
[LibSVM]Training complete with c:  0.021181818181818184 gamma:  2.2224
Model saved as  models/trainedmodel_c_0.021181818181818184_g_2.2224.pkl
[LibSVM]Training complete with c:  0.021181818181818184 gamma:  2.2628
Model saved as  models/trainedmodel_c_0.021181818181818184_g_2.2628.pkl
[LibSVM]Training complete with c:  0.021181818181818184 gamma:  2.3032
Model saved as  models/trainedmodel_c_0.021181818181818184_g_2.3032.pkl
[LibSVM]Training complete with c:  0.021181818181818184 gamma:  2.3436
Model saved as  models/trainedmodel_c_0.021181818181818184_g_2.3436.pkl
[LibSVM]Training complete with c:  0.021181818181818184 gamma:  2.384
Mo

...

[LibSVM]Training complete with c:  0.9798181818181818 gamma:  0.162
Model saved as  models/trainedmodel_c_0.9798181818181818_g_0.162.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  0.2024
Model saved as  models/trainedmodel_c_0.9798181818181818_g_0.2024.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  0.24280000000000002
Model saved as  models/trainedmodel_c_0.9798181818181818_g_0.24280000000000002.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  0.2832
Model saved as  models/trainedmodel_c_0.9798181818181818_g_0.2832.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  0.3236
Model saved as  models/trainedmodel_c_0.9798181818181818_g_0.3236.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  0.364
Model saved as  models/trainedmodel_c_0.9798181818181818_g_0.364.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  0.4044
Model saved as  models/trainedmodel_c_0.9798181818181818_g_0.4044.pkl
[LibS

[LibSVM]Training complete with c:  0.9798181818181818 gamma:  2.586
Model saved as  models/trainedmodel_c_0.9798181818181818_g_2.586.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  2.6264
Model saved as  models/trainedmodel_c_0.9798181818181818_g_2.6264.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  2.6668
Model saved as  models/trainedmodel_c_0.9798181818181818_g_2.6668.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  2.7072
Model saved as  models/trainedmodel_c_0.9798181818181818_g_2.7072.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  2.7476
Model saved as  models/trainedmodel_c_0.9798181818181818_g_2.7476.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  2.788
Model saved as  models/trainedmodel_c_0.9798181818181818_g_2.788.pkl
[LibSVM]Training complete with c:  0.9798181818181818 gamma:  2.8284
Model saved as  models/trainedmodel_c_0.9798181818181818_g_2.8284.pkl
[LibSVM]Training complete with 

[LibSVM]Training complete with c:  0.989909090909091 gamma:  0.8487999999999999
Model saved as  models/trainedmodel_c_0.989909090909091_g_0.8487999999999999.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  0.8891999999999999
Model saved as  models/trainedmodel_c_0.989909090909091_g_0.8891999999999999.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  0.9295999999999999
Model saved as  models/trainedmodel_c_0.989909090909091_g_0.9295999999999999.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  0.97
Model saved as  models/trainedmodel_c_0.989909090909091_g_0.97.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  1.0104
Model saved as  models/trainedmodel_c_0.989909090909091_g_1.0104.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  1.0508
Model saved as  models/trainedmodel_c_0.989909090909091_g_1.0508.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  1.0912
Model saved as  models/trainedmodel_c_0.98

Model saved as  models/trainedmodel_c_0.989909090909091_g_3.3939999999999997.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  3.4343999999999997
Model saved as  models/trainedmodel_c_0.989909090909091_g_3.4343999999999997.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  3.4747999999999997
Model saved as  models/trainedmodel_c_0.989909090909091_g_3.4747999999999997.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  3.5151999999999997
Model saved as  models/trainedmodel_c_0.989909090909091_g_3.5151999999999997.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  3.5555999999999996
Model saved as  models/trainedmodel_c_0.989909090909091_g_3.5555999999999996.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  3.5959999999999996
Model saved as  models/trainedmodel_c_0.989909090909091_g_3.5959999999999996.pkl
[LibSVM]Training complete with c:  0.989909090909091 gamma:  3.6363999999999996
Model saved as  models/trainedmode

[LibSVM]Training complete with c:  1.0 gamma:  2.1012
Model saved as  models/trainedmodel_c_1.0_g_2.1012.pkl
[LibSVM]Training complete with c:  1.0 gamma:  2.1416
Model saved as  models/trainedmodel_c_1.0_g_2.1416.pkl
[LibSVM]Training complete with c:  1.0 gamma:  2.182
Model saved as  models/trainedmodel_c_1.0_g_2.182.pkl
[LibSVM]Training complete with c:  1.0 gamma:  2.2224
Model saved as  models/trainedmodel_c_1.0_g_2.2224.pkl
[LibSVM]Training complete with c:  1.0 gamma:  2.2628
Model saved as  models/trainedmodel_c_1.0_g_2.2628.pkl
[LibSVM]Training complete with c:  1.0 gamma:  2.3032
Model saved as  models/trainedmodel_c_1.0_g_2.3032.pkl
[LibSVM]Training complete with c:  1.0 gamma:  2.3436
Model saved as  models/trainedmodel_c_1.0_g_2.3436.pkl
[LibSVM]Training complete with c:  1.0 gamma:  2.384
Model saved as  models/trainedmodel_c_1.0_g_2.384.pkl
[LibSVM]Training complete with c:  1.0 gamma:  2.4244
Model saved as  models/trainedmodel_c_1.0_g_2.4244.pkl
[LibSVM]Training comple

### Iterate over the trained models

In [37]:
import os
from sklearn.metrics import f1_score

In [38]:
names = os.listdir('models/')

Compute test / train loss for each respective model

In [39]:
test_score = []
train_score = []
for name in names:
    logreg = joblib.load(f'models/{name}')
    preds = logreg.predict(X_test)
    train_preds = logreg.predict(X_train)
    test_score.append(f1_score(y_test, preds))
    train_score.append(f1_score(y_train, train_preds))
    
d = {'name' : names, 'test_score' : test_score, 'train_score' : train_score}

scores = pd.DataFrame(d)

C:\Users\Panu\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Check the result

In [40]:
scores['train_score'].max()

0.8893805309734513

In [41]:
scores['test_score'].max()

0.7461139896373057

In [42]:
scores['test_score'].argmax()

C:\Users\Panu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  """Entry point for launching an IPython kernel.


5101

In [47]:
scores.iloc[5101]

name           trainedmodel_c_0.5156363636363637_g_0.04079999...
test_score                                              0.746114
train_score                                             0.759551
Name: 5101, dtype: object

### Evaluate

Load the model that got the best test set loss

In [48]:
logreg = joblib.load('models/trainedmodel_c_0.5156363636363637_g_0.040799999999999996.pkl')

In [49]:
preds = logreg.predict(X_test)

In [50]:
preds.shape

(267,)

In [51]:
from sklearn.metrics import classification_report

In [52]:
print(classification_report(y_test, preds))

             precision    recall  f1-score   support

          0       0.84      0.87      0.86       167
          1       0.77      0.72      0.75       100

avg / total       0.81      0.82      0.82       267



### Check for overfitting

In [53]:
preds_train = logreg.predict(X_train)

In [54]:
preds_train.shape

(622,)

In [55]:
print(classification_report(y_train, preds_train))

             precision    recall  f1-score   support

          0       0.83      0.91      0.87       382
          1       0.82      0.70      0.76       240

avg / total       0.83      0.83      0.82       622



Our model performs quite nicely! Time to submit results to kaggle

### Import the kaggle test data

In [56]:
val_data = pd.read_csv('test.csv')

In [57]:
val_data.shape

(418, 11)

In [58]:
val_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [59]:
val_passenger_id = val_data['PassengerId']

### Apply the required preprocessing steps

In [60]:
val_data = val_data.drop('Cabin', axis=1)
val_data = val_data.drop(['PassengerId', 'Name', 'Ticket'], axis=1)

In [61]:
val_data.isna().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [62]:
#fill the missing values with mean age/fare
val_data['Age'] = val_data['Age'].fillna(data['Age'].mean())
val_data['Fare'] = val_data['Fare'].fillna(data['Fare'].mean())

In [63]:
val_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [64]:
val_data.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [65]:
val_data[cat_feats] = val_data[cat_feats].astype('object')

In [66]:
val_data_cat_onehot = pd.get_dummies(val_data[cat_feats])

In [67]:
X_val = pd.concat([val_data[num_feats], val_data_cat_onehot], axis=1)

In [68]:
X_val[:] = sc.fit_transform(X_val[:])

In [69]:
X_val.shape

(418, 25)

In [70]:
X_val.head()

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,...,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Parch_9,Embarked_C,Embarked_Q,Embarked_S
0,0.345019,-0.498253,-0.586559,-0.534933,0.957826,-0.755929,0.755929,0.690675,-0.597614,-0.186154,...,-0.37693,-0.29277,-0.085023,-0.069338,-0.04897,-0.04897,-0.069338,-0.568142,2.843757,-1.350676
1,1.335362,-0.513120,-0.586559,-0.534933,0.957826,1.322876,-1.322876,-1.447859,1.673320,-0.186154,...,-0.37693,-0.29277,-0.085023,-0.069338,-0.04897,-0.04897,-0.069338,-0.568142,-0.351647,0.740370
2,2.523773,-0.464935,-0.586559,1.869391,-1.044031,-0.755929,0.755929,0.690675,-0.597614,-0.186154,...,-0.37693,-0.29277,-0.085023,-0.069338,-0.04897,-0.04897,-0.069338,-0.568142,2.843757,-1.350676
3,-0.249187,-0.483313,-0.586559,-0.534933,0.957826,-0.755929,0.755929,0.690675,-0.597614,-0.186154,...,-0.37693,-0.29277,-0.085023,-0.069338,-0.04897,-0.04897,-0.069338,-0.568142,-0.351647,0.740370
4,-0.645324,-0.418318,-0.586559,-0.534933,0.957826,1.322876,-1.322876,-1.447859,1.673320,-0.186154,...,2.65301,-0.29277,-0.085023,-0.069338,-0.04897,-0.04897,-0.069338,-0.568142,-0.351647,0.740370


In [71]:
list(X_val)

['Age',
 'Fare',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'Sex_female',
 'Sex_male',
 'SibSp_0',
 'SibSp_1',
 'SibSp_2',
 'SibSp_3',
 'SibSp_4',
 'SibSp_5',
 'SibSp_8',
 'Parch_0',
 'Parch_1',
 'Parch_2',
 'Parch_3',
 'Parch_4',
 'Parch_5',
 'Parch_6',
 'Parch_9',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

### Make predictions

In [72]:
val_preds = logreg.predict(X_val)

In [73]:
list(val_preds)

[0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,


In [74]:
list(val_passenger_id)

[892,
 893,
 894,
 895,
 896,
 897,
 898,
 899,
 900,
 901,
 902,
 903,
 904,
 905,
 906,
 907,
 908,
 909,
 910,
 911,
 912,
 913,
 914,
 915,
 916,
 917,
 918,
 919,
 920,
 921,
 922,
 923,
 924,
 925,
 926,
 927,
 928,
 929,
 930,
 931,
 932,
 933,
 934,
 935,
 936,
 937,
 938,
 939,
 940,
 941,
 942,
 943,
 944,
 945,
 946,
 947,
 948,
 949,
 950,
 951,
 952,
 953,
 954,
 955,
 956,
 957,
 958,
 959,
 960,
 961,
 962,
 963,
 964,
 965,
 966,
 967,
 968,
 969,
 970,
 971,
 972,
 973,
 974,
 975,
 976,
 977,
 978,
 979,
 980,
 981,
 982,
 983,
 984,
 985,
 986,
 987,
 988,
 989,
 990,
 991,
 992,
 993,
 994,
 995,
 996,
 997,
 998,
 999,
 1000,
 1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023,
 1024,
 1025,
 1026,
 1027,
 1028,
 1029,
 1030,
 1031,
 1032,
 1033,
 1034,
 1035,
 1036,
 1037,
 1038,
 1039,
 1040,
 1041,
 1042,
 1043,
 1044,
 1045,
 1046,
 1047,
 1048,
 1049,
 1

In [75]:
d = {'PassengerId' : list(val_passenger_id), 'Survived' : list(val_preds)}

In [76]:
final_val_preds = pd.DataFrame(d)

In [77]:
final_val_preds.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [78]:
final_val_preds.to_csv('final_val_preds_090918_2.csv', index=False)